# Projeto Prático 1 - Visualização de Malhas

Alunos:
 - Matheus Ventura de Sousa - 11345541
 - Luisa Balleroni          - 

# 1. Introdução

O objetivo deste notebook é apresentar o uso do módulo `Polyscope` da linguagem de Programação Python para renderizar imagens formadas por núvens de pontos. Neste trabalho, será construída uma núvem de pontos para renderizar uma imagem a partir dela. Além disso, é possível usar uma função `callback` para realizar animações visto que a imagem possui uma taxa de atualização. A função callback está definida na seçao <b>1.2. Códigos Auxiliares</b>.

## 1.1 Módulos necessários

In [ ]:
# If you run in colab, run this cell
%%writefile requirements.txt

igl
numpy
polyscope

In [1]:
import igl
import time
import subprocess
import numpy as np
import polyscope as ps
from __future__ import annotations

## 2 Implementação das transformações

O Código abaixo define uma classe `Mesh` usada para encapsular os objetos renderizados pelo `polyscope`. Neste código estão definidas as transformações de `translação`, `rotação`, `escala`, `espelhamento` e `cisalhamento`.

A operação de `translação` tem o objetivo de deslocar o objeto de um lugar para o outro dentro do sistema de coordenadas do mundo. Para estabelecer os novos pontos transladados, usa-se a seguinte matriz:

$$ T(t_x, t_y, t_z) = \begin{bmatrix} 
    1 & 0 & 0 & t_x \\
    0 & 1 & 0 & t_y \\
    0 & 0 & 1 & t_z \\
    0 & 0 & 0 &   1 \\
\end{bmatrix} $$

Onde $t_x, t_y, t_z \in \mathbb{R}$ são coeficientes de translação dos eixos $x, y$ e $z$ respectivamente.

Como o próprio nome diz, a operação de `rotação` rotaciona os pontos dentro do sistema de coordenadas do mundo. Como a rotação é definida dependendo do eixo fixado, foi estabelecida uma nova forma de rotacionar para este trabalho. Considere $x, y$ e $z$ variáveis booleanas que definem se o eixo está fixado na operação. Assim, a partir da definição condicional da rotação, podemos chegar em uma definição genérica:

$$
R(x, y, z, \theta) = 
\begin{bmatrix}
    x + \cos(\theta) (y+z) & \sin(\theta) * (-z)     & \sin(\theta) * (y)     & 0 \\
    \sin(\theta) * (z)     & y + \cos(\theta) *(x+z) & \sin(\theta) * (-x)    & 0 \\
    \sin(\theta) * (-y)    & \sin(\theta) * (x)      & z + \cos(\theta) (x+y) & 0 \\
    0                      & 0                       & 0                      & 1
\end{bmatrix}
$$

Onde $x + y + z = 1$. Observe que, ao fixar qualquer uma das componentes, obtem-se a rotação relativa tal componente fixada!

A transformação em `escala` tem o objetivo de alterar o tamanho do objeto renderizado. Segue a matriz definida para a operação de escala:

$$
S(s_x, s_y, s_z) = 
\begin{bmatrix}
    s_x & 0   & 0   & 0 \\
    0   & s_y & 0   & 0 \\
    0   & 0   & s_z & 0 \\
    0   & 0   & 0   & 1
\end{bmatrix}
$$

O `espelhamento` realiza a inversão de coordenadas comparado ao sistema de coordenadas do mundo. O espelhamento do objeto também depende da coordenada a ser fixada. Visto isso, também será apresentada uma solução genérica para o caso. Considere $x, y$ e $z$ variáveis binárias tal que $x, y, z = 1$ ou $-1$ e que $x + y + z = 1$. A componente fixada deve ser marcada com o valor $-1$. Assim, podemos chegar na seguinte matriz:

Para espelhar, utiliza-se a seguinte matriz:

$$
E(x, y, z) = 
\begin{bmatrix}
    x & 0 & 0 & 0 \\
    0 & y & 0 & 0 \\
    0 & 0 & z & 0 \\
    0 & 0 & 0 & 1
\end{bmatrix}
$$

Para finalizar, a operação de `cisalhamento` reduz a dimensão do objeto para a coordenada de referência em um ponto desejado dentro do sistema de coordenadas do mundo. Seus parâmetros são referentes aos coeficientes de `cisalhamento` para cada par de eixos i,j $(sh_{ij})$. Novamente, esta operação também depende de uma referência, então será apresentada uma solução genérica. Considere $x, y$ e $z$ variáveis binárias tal que $x, y, z = 0$ ou $1$ e $x + y + z = 1$. Dessa forma, a matriz de `cisalhamento` obtem o seguinte padrão:

$$
Sh(x, y, z) =
\begin{bmatrix}
    1               & sh_{yx} \cdot y  & sh_{zx} \cdot z & -(sh_{yx} \cdot y \cdot y_{ref} + sh_{zx} \cdot z \cdot z_{ref}) \\
    sh_{xy} \cdot x & 1                & sh_{zy} \cdot z & -(sh_{xy} \cdot x \cdot x_{ref} + sh_{zy} \cdot z \cdot z_{ref}) \\
    sh_{xz} \cdot x & sh_{yz} \cdot y  & 1               & -(sh_{xz} \cdot x \cdot x_{ref} + sh_{yz} \cdot y \cdot y_{ref}) \\
    0               & 0                & 0               & 1                                                                \\
\end{bmatrix}

$$

In [2]:
class Mesh:
    def __init__(self, vertices: np.ndarray, faces: np.ndarray, mesh_name: str) -> None:
        self.faces = faces
        self.vertices = vertices
        self.mesh_name = mesh_name
        self.mesh_object = ps.register_surface_mesh(mesh_name, vertices, faces) if mesh_name != "" else None

    def translate(self, tx: float, ty: float, tz: float, mesh_name: str) -> Mesh:
        points = np.vstack((
            np.hstack((self.vertices, np.ones((self.vertices.shape[0], 1)))), 
            np.ones((1, self.vertices.shape[1]+1))
        )).T

        translation = np.array([
                [1, 0, 0, tx],
                [0, 1, 0, ty],
                [0, 0, 1, tz],
                [0, 0, 0,  1]
            ])

        return Mesh((translation @ points).T[:-1, :-1], self.faces, mesh_name)

    def rotate(self, fixed_axis: str, theta: float, mesh_name: str) -> Mesh:
        x, y, z = int(fixed_axis == 'x'), int(fixed_axis == 'y'), int(fixed_axis == 'z')

        points = np.vstack((
            np.hstack((self.vertices, np.ones((self.vertices.shape[0], 1)))), 
            np.ones((1, self.vertices.shape[1]+1))
        )).T

        rotation = np.array([
            [1*x + np.cos(theta) * (y+z), np.sin(theta) * (-z), np.sin(theta) * (y), 0],
            [np.sin(theta) * (z), 1*y + np.cos(theta) * (x+z), np.sin(theta) * (-x), 0],
            [np.sin(theta) * (-y), np.sin(theta) * (x), 1*z + np.cos(theta) * (x+y), 0],
            [                  0,                    0,                           0, 1]
        ])

        return Mesh((rotation @ points).T[:-1, :-1], self.faces, mesh_name)
    

    def scale(self, sx:float, sy: float, sz: float, mesh_name: str) -> Mesh:
        points = np.vstack((
            np.hstack((self.vertices, np.ones((self.vertices.shape[0], 1)))), 
            np.ones((1, self.vertices.shape[1]+1))
        )).T

        scale = np.array([
            [sx,  0,  0, 0],
            [ 0, sy,  0, 0],
            [ 0,  0, sz, 0],
            [ 0,  0,  0, 1]
        ])

        return Mesh((scale @ points).T[:-1, :-1], self.faces, mesh_name)

    def reflect(self, inversion_axis: str, mesh_name: str) -> Mesh:
        points = np.vstack((
            np.hstack((self.vertices, np.ones((self.vertices.shape[0], 1)))), 
            np.ones((1, self.vertices.shape[1]+1))
        )).T

        reflection = np.zeros((4,4))
        np.fill_diagonal(reflection, np.array([-1, 1, 1, 1] if inversion_axis == 'x' else [1, -1, 1, 1] if inversion_axis == 'y' else [1, 1, -1, 1]))

        return Mesh((reflection @ points).T[:-1, :-1], self.faces, mesh_name)
    
    def shear(self, mesh_name:str, **kwargs) -> Mesh:
        """
        Esta função aplica uma transformação de cisalhamento em um conjunto de pontos 3D.
        Dependendo do eixo de referência, é necessário passar parâmetros adicionais.
rotate
        Params:
            - reference_axis: strxz
                Eixo de referência da transformação. Pode ser: 'x', 'y' ou 'z'.
            - sh_ij (i, j = {x, y, z} e i != j): float
                Fatores de cisalhamento entre os eixos i e j.
            - x_ref, y_ref, z_ref: float
                Ponto de referência do eixo de referência.

        Returns:
            - Mesh: Objeto Mesh com os pontos transformados. 
        """

        reference_axis = kwargs.get('reference_axis', 'z')
        x, y, z = int(reference_axis == 'x'), int(reference_axis == 'y'), int(reference_axis == 'z') 
        sh_xz, sh_xy, x_ref = (kwargs.get('sh_xz', 0), kwargs.get('sh_xy', 0), kwargs.get('x_ref', 0))
        sh_yx, sh_yz, y_ref = (kwargs.get('sh_yx', 0), kwargs.get('sh_yz', 0), kwargs.get('y_ref', 0))
        sh_zx, sh_zy, z_ref = (kwargs.get('sh_zx', 0), kwargs.get('sh_zy', 0), kwargs.get('z_ref', 0)) 

        points = np.vstack((
            np.hstack((self.vertices, np.ones((self.vertices.shape[0], 1)))), 
            np.ones((1, self.vertices.shape[1]+1))
        )).T

        shear = np.array([
            [        1, sh_yx * y, sh_zx * z, -(sh_yx * y * y_ref + sh_zx * z * z_ref)],
            [sh_xy * x,         1, sh_zy * z, -(sh_xy * x * x_ref + sh_zy * z * z_ref)],
            [sh_xz * x, sh_yz * y,         1, -(sh_xz * x * x_ref + sh_yz * y * y_ref)],
            [        0,         0,         0,                                        1]
        ])

        return Mesh((shear @ points).T[:-1, :-1], self.faces, mesh_name)

# 3. Aplicação da Implementação

## 3.1 Carregamento dos Meshs

Inicialmente, faremos o carregamento dos meshs armazenados em arquivos `.obj` localizados na pasta `./meshes`. Os nomes dos arquivos serão buscados via o comando. Para isso, usaremos o módulo Python `igl`. Cada mesh terá as variações transladadas, rotacionadas, escaladas, refletidas e cisalhadas.

In [5]:
# Carregamento dos Meshs
ps.init()

result = subprocess.run("find . -iname '*.obj'", capture_output=True, shell=True)

mesh_files = result.stdout.decode('utf-8').split('\n')[:-1]
stderr = result.stderr.decode('utf-8').split('\n')

print(mesh_files)

meshes = []

for i, mesh_file in enumerate(mesh_files):
    vertices, _, _, faces, _, _ = igl.read_obj(mesh_file)

    mesh = Mesh(vertices, faces, "").scale(0.1, 0.1, 0.1, "").translate(-20, 0, i*30, "") \
           if mesh_file.find("Rmk3") > 0 else Mesh(vertices, faces, "").translate(-20, 0, i*30, mesh_file.replace('.obj', ''))
    
    rotated_mesh = mesh.translate(20, 0, 0, "").rotate('y', np.pi/3, mesh_file.replace('.obj', '') + "2")
    scaled_mesh = mesh.translate(40, 0, 0, "").scale(0.3, 0.3, 0.3, mesh_file.replace('.obj', '') + "3") if mesh_file.find("Rmk3") > 0 \
                  else mesh.translate(60, 0, 0, "").scale(0.7, 0.7, 0.7, mesh_file.replace('.obj', '') + "3")
    reflected_mesh = mesh.reflect('y', mesh_file.replace('.obj', '') + "4")
    sheared_mesh = rotated_mesh.shear(mesh_file.replace('.obj', '') + "5", reference_axis='y', sh_yx=2, sh_yz=0, y_ref=0)

    meshes.append([mesh, rotated_mesh, scaled_mesh, reflected_mesh, sheared_mesh])

ps.show()

['./meshes/FinalBaseMesh.obj', './meshes/indoor plant_02.obj', './meshes/Rmk3.obj']


  o indoor_plant_02
  o Plane_Plane.001


Também é possível armazenar a renderização em uma imagem `.png` ou `.jpg`, como faz a celula a seguir:

In [6]:
OUTPUT_PATH = "./images/polyscope_image.png"

_ = subprocess.run("mkdir -p images", capture_output=True, shell=True)
ps.look_at((250, 100, 250), (50, 0,30))
ps.screenshot(filename=OUTPUT_PATH, transparent_bg=True)

# 4. Conclusão

O presente trabalho apresentou como o módulo `polyscope` é usado e como são aplicadas as principais transformações em cima das malhas importadas. Um ponto importante foi a conclusão de matrizes genéricas para a implementação das transformações `rotação` e `cisalhamento`, visto que houve grande simplificação para as funções. Além disso, foi importante para compreender como as transformações afetam as malhas de maneira intuitiva. 